In [275]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
from pandas import json_normalize
import time
import sys
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd


To verify the best location, we need to verify each venue in Foursquare or through external APIs:

# 1. Airports

In [281]:
df=pd.read_csv("C:/Users/Leticia Martinez/Desktop/IRONHACK/Labs/Project_III/data/df4.csv")


In [282]:
df_airport=pd.read_csv("C:/Users/Leticia Martinez/Desktop/IRONHACK/Labs/Project_III/data/airport_code.csv")
df_airport = df_airport.drop(["elevation_ft", "continent","iso_region","scheduled_service","municipality","scheduled_service","gps_code","local_code"], axis=1)
df_airport = df_airport[df_airport['iso_country'] == 'US']
df_airport = df_airport[df_airport['type'] == 'large_airport']
df_airport.sample()

,id,ident,type,name,latitude_deg,longitude_deg,iso_country
28341,3454,KCLE,large_airport,Cleveland Hopkins International Airport,41.411701,-81.8498,US


In [285]:
from geopy.distance import geodesic
import pandas as pd
df['aeropuerto_cercano'] = ''
df['diferencia_km'] = 0.0
for index, row in df.iterrows():
    lat1 = row['latitude']  
    lon1 = row['longitude']  
    min_distance = float('inf')
    aeropuerto_cercano = ''
    
    for idx, r in df_airport.iterrows():
        lat2 = r['latitude_deg'] 
        lon2 = r['longitude_deg']  
        
        distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
        
        if distance < min_distance:
            min_distance = distance
            aeropuerto_cercano = r['name'] 
    
    lat2 = df_airport.loc[df_airport['name'] == aeropuerto_cercano, 'latitude_deg'].values[0]
    lon2 = df_airport.loc[df_airport['name'] == aeropuerto_cercano, 'longitude_deg'].values[0]
    diferencia_metros = geodesic((lat1, lon1), (lat2, lon2)).meters
    
    df.at[index, 'aeropuerto_cercano'] = aeropuerto_cercano
    df.at[index, 'diferencia_metros'] = diferencia_metros

df.sample()


,Unnamed: 0,name,description,number_of_employees,total_money_raised,category_code,tag_list,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,aeropuerto_cercano,diferencia_km,diferencia_metros
49,249,Comixology,Main Office,NaN,$3.47M,games_video,NaN,159 West 25th Street 9th floor,NaN,10010-4436,New York,NY,USA,40.740512,-73.987794,La Guardia Airport,0.0,10546.064717


In [284]:
subset1 = df[df['state_code'] == 'NY']
subset = df[df['state_code'] == 'CA']

Extracting the airports just from San Francisco close to our middle point:

In [216]:
import pandas as pd
airports = ['Metropolitan Oakland International Airport',
       'Norman Y. Mineta San Jose International Airport',
       'Los Angeles International Airport',
       'San Francisco International Airport',
       'Bangor International Airport']

subset_airport = df_airport[df_airport['name'].isin(airports)][['name', 'latitude_deg', 'longitude_deg']]
df_airport = pd.DataFrame(subset_airport)
df_airport.to_csv('./data/df_airports.csv', index=False)
df_airport

Extracting the airports just from NY close to our middle point:

In [287]:
subset1["aeropuerto_cercano"].unique()

array(['La Guardia Airport', 'Tulsa International Airport',
       'John F Kennedy International Airport'], dtype=object)

In [288]:
import pandas as pd
airports = ['La Guardia Airport', 'Tulsa International Airport',
       'John F Kennedy International Airport']

subset_airportny = df_airport[df_airport['name'].isin(airports)][['name', 'latitude_deg', 'longitude_deg']]
df_airport_NY = pd.DataFrame(subset_airportny)
df_airport_NY.to_csv('./data/df_airports_ny.csv', index=False)

# 2. 1 Starbucks - San Francisco

In [220]:
token = os.getenv("token")
load_dotenv()

In [221]:
load_dotenv()

True

In [222]:
where="San Francisco"
SF_long=-122.40045773333333
SF_lat=37.781981995833334

In [223]:
import requests
url='https://api.foursquare.com/v3/places/search?query=starbucks' 
headers = {
    "accept": "application/json",
    "Authorization": token
}
response = requests.get(url, headers=headers)

In [224]:
def Starbucks_SF (query, SF_long, SF_lat, radius, limit):       
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={SF_long}%2C{SF_lat}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [225]:
response= Starbucks_SF("starbucks", 37.781981995833334, -122.40045773333333, 800, 20)

In [226]:
def everything_SFST (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    latitude= one_element["geocodes"]["roof"]["latitude"]
    longitude= one_element["geocodes"]["roof"]["longitude"]
    dict1 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude}
    
    return dict1

In [227]:
everything_SFST (response["results"][1])

{'name': 'Starbucks',
 'address': '780 Market St (at 4th St), San Francisco, CA 94102',
 'latitude': 37.786145,
 'longitude': -122.405612}

In [228]:
new_listSFST = []
for i in response["results"]:
    new_listSFST.append(everything_SFST (i))

df_SF_ST = pd.DataFrame(new_listSFST)
df_SF_ST
     

,name,address,latitude,longitude
0,Starbucks,"865 Market St (5th street), San Francisco, CA ...",37.784117,-122.407356
1,Starbucks,"780 Market St (at 4th St), San Francisco, CA 9...",37.786145,-122.405612
2,Starbucks,"120 4th St (at Mission St), San Francisco, CA ...",37.784199,-122.404135
3,Starbucks,"74 New Montgomery St (at Jessie St), San Franc...",37.787664,-122.401113
4,Starbucks,"201 3rd St, San Francisco, CA 94103",37.785019,-122.399810


In [229]:
df_SF_ST.to_csv('./data/df_SF_starbucks.csv')


# 2.2 Schools San Francisco

In [289]:
def Schools_SF (SF_long, SF_lat, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?ll={SF_lat}%2C{SF_long}&radius={radius}&categories=12056,12057,12058"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [290]:
response= Schools_SF(-122.40045773333333, 37.781981995833334, 500)


In [291]:
def everything_SFSC (one_element):    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["roof"]["latitude"]
    longitude= one_element["geocodes"]["roof"]["longitude"]
    dict2 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict2

In [234]:
new_listSFSC = []
for i in response["results"]:
    new_listSFSC.append(everything_SFSC (i))

df_SF_SC = pd.DataFrame(new_listSFSC)
df_SF_SC

,name,address,latitude,longitude,categories
0,AltSchool Yerba Buena,"300 4th St, San Francisco, CA 94107",37.781777,-122.401196,Elementary School
1,KinderCare Learning Center,"75 Hawthorne St, San Francisco, CA 94105",37.785486,-122.397974,Preschool
2,Bessie Carmichael Elementary,"824 Harrison St, San Francisco, CA 94107",37.780661,-122.400414,Primary and Secondary School
3,San Francisco Unified School District,"824 Harrison St, San Francisco, CA 94107",37.780634,-122.400340,Primary and Secondary School
4,San Francisco Unified School District,"825 Shotwell St, San Francisco, CA 94110",37.780596,-122.400410,Elementary School
5,Fec Middle School,"824 Harrison St, San Francisco, CA 94107",37.780504,-122.400333,Primary and Secondary School
6,California Charter Schools,"730 Harrison St, San Francisco, CA 94107",37.782152,-122.398423,Primary and Secondary School
7,SOMA Daycare & Preschool,"1112 Howard St, San Francisco, CA 94107",37.781318,-122.395649,Preschool


In [235]:
df_SF_SC.to_csv('./data/df_SF_schools.csv')


# 2.3 Vegan restaurants - San Francisco 

In [236]:
def vegan_SF (SF_long, SF_lat, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?ll={SF_lat}%2C{SF_long}&radius={radius}&categories=13377"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    
    try:
        places = response["results"]  # Revisa la estructura de la respuesta JSON
    except KeyError:
        places = []
    
    vegan_places = []
    for place in places:
        categories = place["categories"]
        category_ids = [category["id"] for category in categories]
        if len(category_ids) == 1 and 13377 in category_ids:
            vegan_places.append(place)
    
    return vegan_places

In [237]:
vegan_places= vegan_SF(-122.40045773333333, 37.781981995833334, 1000)


In [238]:
def everything_SFVG (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["roof"]["latitude"]
    longitude= one_element["geocodes"]["roof"]["longitude"]
    dict2 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict2

In [240]:
new_listSFVG = []
for i in vegan_places:
    new_listSFVG.append(everything_SFVG (i))

df_SF_VG = pd.DataFrame(new_listSFVG)
df_SF_VG

,name,address,latitude,longitude,categories
0,Plant B,"60 Morris St, San Francisco, CA 94107",37.776757,-122.402764,Vegan and Vegetarian Restaurant
1,RawDaddy's Fun Cone Food,"1025 Market St, San Francisco, CA 94103",37.781373,-122.410779,Vegan and Vegetarian Restaurant


In [241]:
df_SF_VG.to_csv('./data/df_SF_veganrestaurants.csv')


# 2.4 Pet grooming service - San Francisco

In [242]:
def Pets_SF (SF_long, SF_lat, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=11134&ll={SF_lat}%2C{SF_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

    return response

In [243]:
response= Pets_SF(-122.40045773333333, 37.781981995833334, 800)

In [244]:
def everything_SFPet (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["roof"]["latitude"]
    longitude= one_element["geocodes"]["roof"]["longitude"]
    dict3 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict3

In [246]:
new_listSFPet = []
for i in response["results"]:
    new_listSFPet.append(everything_SFPet (i))

df_SF_Pet = pd.DataFrame(new_listSFPet)
df_SF_Pet

,name,address,latitude,longitude,categories
0,SF Puppy Love,"840 Harrison St (4th), San Francisco, CA 94107",37.779739,-122.401363,Pet Grooming Service
1,Doggie Day Spaw,"920 Harrison St, San Francisco, CA 94107",37.778860,-122.402602,Pet Grooming Service
2,Tefani & So. Dogcare,"156 2nd St, San Francisco, CA 94105",37.787063,-122.399142,Pet Grooming Service


In [247]:
df_SF_Pet.to_csv('./data/df_SF_pets.csv')


# 2.4 Party - San Francisco ( nightclubs, pubs, beer bars)

In [248]:
def Partying_SF (SF_long, SF_lat, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=13009,13007,13006,13015,13018&ll={SF_lat}%2C{SF_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

    return response

In [293]:
response= Partying_SF(-122.40045773333333, 37.781981995833334, 800)


In [250]:
def everything_SFParty (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["roof"]["latitude"]
    longitude= one_element["geocodes"]["roof"]["longitude"]
    dict4 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict4

In [252]:
new_listSFParty = []
for i in response["results"]:
    new_listSFParty.append(everything_SFParty (i))

df_SF_Party = pd.DataFrame(new_listSFParty)
df_SF_Party

,name,address,latitude,longitude,categories
0,Black Hammer Brewing,"544 Bryant St (at 3rd St), San Francisco, CA 9...",37.780716,-122.397119,Beer Garden
1,Local Edition,"691 Market St (at 3rd St), San Francisco, CA 9...",37.787577,-122.403115,Cocktail Bar
2,Wine Down SF,"685 Folsom St (3rd St.), San Francisco, CA 94107",37.783984,-122.398387,Beer Bar
3,Taksim,"564 4th St, San Francisco, CA 94107",37.778732,-122.397433,Cocktail Bar
4,Kona's Street Market,"32 3rd St, San Francisco, CA 94103",37.786996,-122.403440,Cocktail Bar
5,Rooh,"333 Brannan St, San Francisco, CA 94107",37.781075,-122.392409,Cocktail Bar
6,Marianne's,"360 Jessie St (at 5th St), San Francisco, CA 9...",37.783499,-122.406890,Cocktail Bar
7,Novela,"662 Mission St (at Annie St), San Francisco, C...",37.787122,-122.401368,Cocktail Bar
8,Burma Club,"8 Mint Plz, San Francisco, CA 94103",37.782957,-122.408005,Cocktail Bar
9,Gold Club,"650 Howard St, San Francisco, CA 94105",37.785979,-122.399652,Cocktail Bar


In [253]:
df_SF_Party.to_csv('./data/df_SF_party.csv')


# 2. 5 Basket Stadium - San Francisco

In [254]:
def Basket_SF (SF_long, SF_lat, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=18008&ll={SF_lat}%2C{SF_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

    return response

In [255]:
response= Basket_SF(-122.40045773333333, 37.781981995833334, 3000)


In [256]:
def everything_SFbasket(one_element):
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category = one_element["categories"][0]["name"]
    latitude = one_element["geocodes"]["main"]["latitude"]
    longitude = one_element["geocodes"]["main"]["longitude"]
    dict5 = {"name": name, "address": address, "latitude": latitude, "longitude": longitude, "categories": category}
    return dict5

new_listSFbasket = []
for i in response["results"]:
    new_listSFbasket.append(everything_SFbasket(i))

df_SF_basket = pd.DataFrame(new_listSFbasket)
df_SF_basket


,name,address,latitude,longitude,categories
0,Berry Basketball Courts,"Berry St (I-280), San Francisco, CA 94158",37.772080,-122.397686,Basketball Court
1,Soma Rec Center,"6TH St (Folsom Street), San Francisco, CA 94109",37.778838,-122.406328,Basketball Court
2,NBA Nation,"20TH St, San Francisco, CA 94110",37.759168,-122.412335,Basketball Court
3,Alice Marble Basketball Court,"2201 Hyde St (at Greenwich), San Francisco, CA...",37.801449,-122.420352,Basketball Court


In [257]:
df_SF_basket.to_csv('./data/df_SF_basket.csv')


# 3. All venues - San Francisco

In [258]:
import folium

# Center de map to geolocation
mapa = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

ruta_colegios = './data/df_SF_schools.csv'
df_colegios = pd.read_csv(ruta_colegios)

pets_data = './data/df_SF_pets.csv'
df_pets = pd.read_csv(pets_data)

starbucks_data = './data/df_SF_starbucks.csv'
df_starbucks = pd.read_csv(starbucks_data)

ruta_empresas = './data/df_SF.csv'
df_empresas = pd.read_csv(ruta_empresas)

ruta_restaurantes = './data/df_SF_veganrestaurants.csv'
df_restaurantes = pd.read_csv(ruta_restaurantes)

party = './data/df_SF_party.csv'
df_party = pd.read_csv(party)

airports = './data/df_airports.csv'
df_airports = pd.read_csv(airports)

basket='./data/df_SF_basket.csv'
df_basket=pd.read_csv(basket)

# Create markers for each category:

'''basket'''
for index, row in df_basket.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='orange', icon='hand-peace', prefix = "fa"),
                  popup=popup_content).add_to(mapa)

'''schools'''
for index, row in df_colegios.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='blue', icon='graduation-cap', prefix='fa'),
                  popup=popup_content).add_to(mapa)


'''restaurants'''
for index, row in df_restaurantes.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"

    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='green', icon='leaf'),
                  popup=popup_content).add_to(mapa)

'''companies > 1B, + design companies'''

for index, row in df_empresas.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='red', icon='building', prefix = "fa"),
                  popup=popup_content).add_to(mapa)

    
'''Pet Grooming service'''

for index, row in df_pets.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='pink', icon='paw', prefix = "fa"),
                  popup=popup_content).add_to(mapa)
    
'''Party'''
    
for index, row in df_party.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='black', icon='beer', prefix = "fa"),
                  popup=popup_content).add_to(mapa)
    
    
'''Starbucks'''
for index, row in df_starbucks.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='purple', icon='coffee', prefix = "fa"),
                  popup=popup_content).add_to(mapa)

'''Airports'''
for index, row in df_airports.iterrows():
    popup_content = f"Latitude: {row['latitude_deg']}<br>Longitude: {row['longitude_deg']}<br>Name: {row['name']}"
    folium.Marker([row['latitude_deg'], row['longitude_deg']],
                  icon=folium.Icon(color='cadetblue', icon='plane', prefix = "fa"),
                  popup=popup_content).add_to(mapa)   
mapa


# 4. Weighing categories

Hereby we have set a weighing scoring system for each category based on their group value: company culture, employee benefits, or expenses. As well the maximum distance allowed is described per requirement.
The weighing scores are defined by the values of the company and their importance and the nº of employees affected by each requirement.

In [297]:
df_hipotesis=pd.read_csv('data/hipotesis.csv')
df_hipotesis.drop("Unnamed: 5", axis=1, inplace=True)
df_hipotesis.drop("Points", axis=1, inplace=True)
df_hipotesis

,Importance,Group,Ponderation,Criteria,Employees,Distance min,Distance max,SOURCE
0,1,Company culture,23.0,Designers like to go to design talks and share...,20,"0,1",500,MongoDB & Foursquare
1,2,Employees benefits,19.0,30% of the company staff have at least 1 child.,26,"0,1",500,Foursquare
2,3,Company culture,15.0,Executives like Starbucks A LOT. Ensure there'...,10,0,800,Foursquare
3,4,Reduce travel expenses,12.0,Account managers need to travel a lot,20,20,20,Foursquare
4,5,Company culture,8.0,The CEO is vegan.,1,"0,1",1000,Foursquare
5,6,Company culture,8.0,"The office dog—""Dobby"" needs a hairdresser eve...",1,"0,1",800,Foursquare
6,6,Company culture,8.0,"Everyone in the company is between 25 and 40, ...",87,"0,1",800,Foursquare
7,7,Company culture,7.0,"If you want to make the maintenance guy happy,...",1,2,10000,Foursquare
8,MANDATORY,Strategy / Position,NaN,Developers like to be near successful tech sta...,15,Nan,NAN,MongoDB


We iterate the location A with the requirements ( weigh & distance)

In [261]:
from geopy.distance import geodesic

#Potencial location A
locations = [
    {"name": "A", "latitude": 37.785077, "longitude": -122.400605}, #(3rd avenida/mission st)
]

# requirements and weighs.

criteriaA = [
    {"name": "Requirement 1", "ponderation": 0.23, "distance_max": 500, "latitude": 37.786942, "longitude": -122.401245},
    {"name": "Requirement 2", "ponderation": 0.19, "distance_max": 500, "latitude": 37.785486, "longitude": -122.397974},
    {"name": "Requirement 3", "ponderation": 0.15, "distance_max": 800, "latitude": 37.787664, "longitude": -122.401113},
    {"name": "Requirement 4", "ponderation": 0.12, "distance_max": 30000, "latitude": 37.6189994812012, "longitude": -122.375},
    {"name": "Requirement 5", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.781373, "longitude": -122.410779},
    {"name": "Requirement 6", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.787063, "longitude": -122.399142},
    {"name": "Requirement 7", "ponderation": 0.08, "distance_max": 500, "latitude": 37.787122, "longitude": -122.401368},
    {"name": "Requirement 8", "ponderation": 0.08, "distance_max": 10000, "latitude": 37.778838, "longitude": -122.406328},
]


# Calculate the score and distance of each location for each requirement
# Calculate the degree of compliance for each requirement by geodesic
data = []
for criterion in criteriaA:
    requirement_name = criterion["name"]
    ponderation = criterion["ponderation"]
    distance_max = criterion["distance_max"]
    latitude = criterion["latitude"]
    longitude = criterion["longitude"]
    distance = geodesic((initial_location["latitude"], initial_location["longitude"]), (latitude, longitude)).meters
    
    # Calculate the compliance based on distance
    if distance <= distance_max:
        compliance_score = 1.0
    else:
        compliance_score = 0.0
    
    row = {"Requirement": requirement_name, "Ponderation A": ponderation, "Distance A": distance, "Compliance A": compliance_score}
    data.append(row)
df1 = pd.DataFrame(data)
df1["Compliance Score A"] = df1["Ponderation A"] * df1["Compliance A"]
df1


,Requirement,Ponderation A,Distance A,Compliance A,Compliance Score A
0,Requirement 1,0.23,708.958359,0.0,0.00
1,Requirement 2,0.19,806.183145,0.0,0.00
2,Requirement 3,0.15,782.709236,1.0,0.15
3,Requirement 4,0.12,18254.474142,1.0,0.12
4,Requirement 5,0.08,444.854066,1.0,0.08
5,Requirement 6,0.08,836.446039,1.0,0.08
6,Requirement 7,0.08,719.758819,0.0,0.00
7,Requirement 8,0.08,315.712566,1.0,0.08


In [262]:
subset_df1 = df1[["Distance A", "Compliance Score A"]]
subset_df1

,Distance A,Compliance Score A
0,708.958359,0.00
1,806.183145,0.00
2,782.709236,0.15
3,18254.474142,0.12
4,444.854066,0.08
5,836.446039,0.08
6,719.758819,0.00
7,315.712566,0.08


We iterate the location B with the requirements ( weigh & distance)

In [263]:
from geopy.distance import geodesic
import pandas as pd

initial_location = {"latitude": 37.781644, "longitude": -122.4057407}  # 5a avenida

criteriaB = [
    {"name": "Requirement 1", "ponderation": 0.23, "distance_max": 500, "latitude": 37.781557, "longitude": -122.407959},
    {"name": "Requirement 2", "ponderation": 0.19, "distance_max": 500, "latitude": 37.781777, "longitude": -122.401196},
    {"name": "Requirement 3", "ponderation": 0.15, "distance_max": 800, "latitude": 37.784117, "longitude": -122.407356},
    {"name": "Requirement 4", "ponderation": 0.12, "distance_max": 30000, "latitude": 37.6189994812012, "longitude": -122.375},
    {"name": "Requirement 5", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.781373, "longitude": -122.410779},
    {"name": "Requirement 6", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.77886, "longitude": -122.402602},
    {"name": "Requirement 7", "ponderation": 0.08, "distance_max": 10000, "latitude": 37.783499, "longitude": -122.40689},
    {"name": "Requirement 8", "ponderation": 0.08, "distance_max": 10000, "latitude": 37.778838, "longitude": -122.406328},
]

data = []
for criterion in criteriaB:
    requirement_name = criterion["name"]
    ponderation = criterion["ponderation"]
    distance_max = criterion["distance_max"]
    latitude = criterion["latitude"]
    longitude = criterion["longitude"]

    distance = geodesic((initial_location["latitude"], initial_location["longitude"]), (latitude, longitude)).meters
    
    if distance <= distance_max:
        compliance_score = 1.0
    else:
        compliance_score = 0.0
    
    row = {"Requirement": requirement_name, "Ponderation B": ponderation, "Distance B": distance, "Compliance B": compliance_score}
    data.append(row)

df = pd.DataFrame(data)
df["Compliance Score B"] = df["Ponderation B"] * df["Compliance B"]
df


,Requirement,Ponderation B,Distance B,Compliance B,Compliance Score B
0,Requirement 1,0.23,195.653593,1.0,0.23
1,Requirement 2,0.19,400.624562,1.0,0.19
2,Requirement 3,0.15,309.174468,1.0,0.15
3,Requirement 4,0.12,18254.474142,1.0,0.12
4,Requirement 5,0.08,444.854066,1.0,0.08
5,Requirement 6,0.08,414.650526,1.0,0.08
6,Requirement 7,0.08,229.436735,1.0,0.08
7,Requirement 8,0.08,315.712566,1.0,0.08


In [264]:
subset_df = df[["Distance B", "Compliance Score B"]]
subset_df

,Distance B,Compliance Score B
0,195.653593,0.23
1,400.624562,0.19
2,309.174468,0.15
3,18254.474142,0.12
4,444.854066,0.08
5,414.650526,0.08
6,229.436735,0.08
7,315.712566,0.08


We iterate the location C with the requirements ( weigh & distance)

In [268]:
from geopy.distance import geodesic

locations = [

    {"name": "C", "latitude": 37.786873, "longitude": -122.398352} #(2nd avenida/mission st)
]

criteriaC = [
    {"name": "1.Design companies", "ponderation": 0.23, "distance_max": 500, "latitude": 37.788668, "longitude": -122.400558},
    {"name": "2.Schools", "ponderation": 0.19, "distance_max": 500, "latitude": 37.785486, "longitude": -122.404656},
    {"name": "3.Starbucks", "ponderation": 0.15, "distance_max": 800, "latitude": 37.784117, "longitude": -122.397974},
    {"name": "4.Airports ", "ponderation": 0.12, "distance_max": 30000, "latitude": 37.6189994812012, "longitude": -122.375},
    {"name": "5.Vegan Restaurant", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.781373, "longitude": -122.410779},
    {"name": "6.Dogs haidresser", "ponderation": 0.08, "distance_max": 1000, "latitude": 37.787063, "longitude": -122.399142},
    {"name": "7. Pubs", "ponderation": 0.08, "distance_max": 500, "latitude": 37.787122, "longitude": -122.401368},
    {"name": "8. Basketball court", "ponderation": 0.08, "distance_max": 10000, "latitude": 37.778838, "longitude": -122.406328},
]

data = []
for criterion in criteriaC:
    requirement_name = criterion["name"]
    ponderation = criterion["ponderation"]
    distance_max = criterion["distance_max"]
    latitude = criterion["latitude"]
    longitude = criterion["longitude"]

    distance = geodesic((initial_location["latitude"], initial_location["longitude"]), (latitude, longitude)).meters
    
    if distance <= distance_max:
        compliance_score = 1.0
    else:
        compliance_score = 0.0
    
    row = {"Requirement": requirement_name, "Ponderation": ponderation, "Distance C": distance, "Compliance C": compliance_score}
    data.append(row)

df5 = pd.DataFrame(data)
df5["Compliance Score C"] = df5["Ponderation"] * df5["Compliance C"]
df5


,Requirement,Ponderation,Distance C,Compliance C,Compliance Score C
0,1.Design companies,0.23,903.446885,0.0,0.00
1,2.Schools,0.19,437.006768,1.0,0.19
2,3.Starbucks,0.15,737.181494,1.0,0.15
3,4.Airports,0.12,18254.474142,1.0,0.12
4,5.Vegan Restaurant,0.08,444.854066,1.0,0.08
5,6.Dogs haidresser,0.08,836.446039,1.0,0.08
6,7. Pubs,0.08,719.758819,0.0,0.00
7,8. Basketball court,0.08,315.712566,1.0,0.08


In [272]:
concatenated_df = pd.concat([df5, subset_df1, subset_df], axis=1)
column_order = ['Requirement', 'Ponderation', 'Distance A', 'Compliance Score A', 'Distance B', 'Compliance Score B', 'Distance C', 'Compliance C', 'Compliance Score C']
concatenated_df = concatenated_df.reindex(columns=column_order)
concatenated_df

,Requirement,Ponderation,Distance A,Compliance Score A,Distance B,Compliance Score B,Distance C,Compliance C,Compliance Score C
0,1.Design companies,0.23,708.958359,0.00,195.653593,0.23,903.446885,0.0,0.00
1,2.Schools,0.19,806.183145,0.00,400.624562,0.19,437.006768,1.0,0.19
2,3.Starbucks,0.15,782.709236,0.15,309.174468,0.15,737.181494,1.0,0.15
3,4.Airports,0.12,18254.474142,0.12,18254.474142,0.12,18254.474142,1.0,0.12
4,5.Vegan Restaurant,0.08,444.854066,0.08,444.854066,0.08,444.854066,1.0,0.08
5,6.Dogs haidresser,0.08,836.446039,0.08,414.650526,0.08,836.446039,1.0,0.08
6,7. Pubs,0.08,719.758819,0.00,229.436735,0.08,719.758819,0.0,0.00
7,8. Basketball court,0.08,315.712566,0.08,315.712566,0.08,315.712566,1.0,0.08


# 5. Visualization by requirements and locations

In [301]:
import plotly.express as px
import plotly.io as pio
fig = px.bar(concatenated_df, x="Requirement", y=["Compliance Score A", "Compliance Score B", "Compliance Score C"])

fig.update_layout(
    title="Compliance requirements",
    xaxis_title="Requirements",
    yaxis_title="Compliance Score",
    legend_title="Criterias"
)
pio.write_html(fig, file='requirements_locSF.html', auto_open=False)
fig.show()


In [298]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(x=["Location A"], y=[concatenated_df["Compliance Score A"].sum()], name="Compliance Score A"))
fig.add_trace(go.Bar(x=["Location B"], y=[concatenated_df["Compliance Score B"].sum()], name="Compliance Score B"))
fig.add_trace(go.Bar(x=["Location C"], y=[concatenated_df["Compliance Score C"].sum()], name="Compliance Score C"))
fig.update_layout(
    title="Compliance Scores",
    xaxis_title="Locations",
    yaxis_title="Compliance Score",
    barmode="group"
)
pio.write_html(fig, file='compliance_locSF.html', auto_open=False)
fig.show()


In [300]:
df_final=pd.concat([df_hipotesis, concatenated_df], axis=1)
df_final

,Importance,Group,Ponderation,Criteria,Employees,Distance min,Distance max,SOURCE,Requirement,Ponderation,Distance A,Compliance Score A,Distance B,Compliance Score B,Distance C,Compliance C,Compliance Score C
0,1,Company culture,23.0,Designers like to go to design talks and share...,20,"0,1",500,MongoDB & Foursquare,1.Design companies,0.23,708.958359,0.00,195.653593,0.23,903.446885,0.0,0.00
1,2,Employees benefits,19.0,30% of the company staff have at least 1 child.,26,"0,1",500,Foursquare,2.Schools,0.19,806.183145,0.00,400.624562,0.19,437.006768,1.0,0.19
2,3,Company culture,15.0,Executives like Starbucks A LOT. Ensure there'...,10,0,800,Foursquare,3.Starbucks,0.15,782.709236,0.15,309.174468,0.15,737.181494,1.0,0.15
3,4,Reduce travel expenses,12.0,Account managers need to travel a lot,20,20,20,Foursquare,4.Airports,0.12,18254.474142,0.12,18254.474142,0.12,18254.474142,1.0,0.12
4,5,Company culture,8.0,The CEO is vegan.,1,"0,1",1000,Foursquare,5.Vegan Restaurant,0.08,444.854066,0.08,444.854066,0.08,444.854066,1.0,0.08
5,6,Company culture,8.0,"The office dog—""Dobby"" needs a hairdresser eve...",1,"0,1",800,Foursquare,6.Dogs haidresser,0.08,836.446039,0.08,414.650526,0.08,836.446039,1.0,0.08
6,6,Company culture,8.0,"Everyone in the company is between 25 and 40, ...",87,"0,1",800,Foursquare,7. Pubs,0.08,719.758819,0.00,229.436735,0.08,719.758819,0.0,0.00
7,7,Company culture,7.0,"If you want to make the maintenance guy happy,...",1,2,10000,Foursquare,8. Basketball court,0.08,315.712566,0.08,315.712566,0.08,315.712566,1.0,0.08
8,MANDATORY,Strategy / Position,NaN,Developers like to be near successful tech sta...,15,Nan,NAN,MongoDB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 6. New York - research 

## 6.1 Venue

In [177]:
#NY_lat: 40.745147826315794
#NY_long: -73.98944178947369

In [3]:
import requests

url='https://api.foursquare.com/v3/places/search?query=starbucks' 


headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

In [5]:
def Starbucks_NY (query, NY_long, NY_lat, radius, limit):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={NY_long}%2C{NY_lat}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [11]:
response= Starbucks_NY("starbucks", 40.7451478, -73.9894417, 800, 20)


In [9]:
def everything_NYST(one_element):    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    latitude = one_element["geocodes"]["main"]["latitude"]
    longitude = one_element["geocodes"]["main"]["longitude"]
    dict1 = {"name": name, "address": address, "latitude": latitude, "longitude": longitude}
    return dict1  
    return dict1
new_listNYST = []
for i in response["results"]:
    new_listNYST.append(everything_NYST (i))

df_NY_ST = pd.DataFrame(new_listNYST)
df_NY_ST
df_NY_ST.to_csv('./data/df_NY_ST.csv')
    

In [13]:
def Schools_NY (NY_lat, NY_long, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?ll={NY_lat}%2C{NY_long}&radius={radius}&categories=12056,12057,12058"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

In [15]:
response= Schools_NY(40.7451478, -73.9894417,500)


In [16]:
def everything_NYSC (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["main"]["latitude"]
    longitude= one_element["geocodes"]["main"]["longitude"]
    dict2 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict2
new_listNYSC = []
for i in response["results"]:
    new_listNYSC.append(everything_NYSC (i))

df_NY_SC = pd.DataFrame(new_listNYSC)
df_NY_SC
df_NY_SC.to_csv('./data/df_NY_SC.csv')


In [18]:
def Pets_NY (NY_lat,NY_long,radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=11134&ll={NY_lat}%2C{NY_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

    return response

In [20]:
response= Pets_NY(40.7451478, -73.9894417, 800)


In [21]:
def everything_NYPet (one_element):
    
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category=one_element["categories"][0]["name"]
    latitude= one_element["geocodes"]["main"]["latitude"]
    longitude= one_element["geocodes"]["main"]["longitude"]
    dict4 = {"name": name, "address":address, "latitude":latitude, "longitude":longitude, "categories":category}
    
    return dict4
new_listNYPet = []
for i in response["results"]:
    new_listNYPet.append(everything_NYPet (i))

df_NY_Pet = pd.DataFrame(new_listNYPet)
df_NY_Pet
df_NY_Pet.to_csv('./data/df_NY_Pet.csv')


In [27]:
def Basket_NY (NY_lat,NY_long, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=18008&ll={NY_lat}%2C{NY_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

    return response

In [32]:
response= Basket_NY (40.7451478, -73.9894417, 3000)


In [29]:
def everything_Basket_NY(one_element):
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category = one_element["categories"][0]["name"]
    latitude = one_element["geocodes"]["main"]["latitude"]
    longitude = one_element["geocodes"]["main"]["longitude"]
    dict10 = {"name": name, "address": address, "latitude": latitude, "longitude": longitude, "categories": category}
    return dict10

In [31]:
new_listNYbasket = []
for i in response["results"]:
    new_listNYbasket.append(everything_Basket_NY(i))

df_NY_basket = pd.DataFrame(new_listNYbasket)


,name,address,latitude,longitude,categories
0,Basketball Court,"Stuyvesant Walk (18th Street), New York, NY 10009",40.733342,-73.980450,Basketball Court
1,West 4th Street Courts,"W 3RD St (6 Av), New York, NY 10012",40.731534,-74.001380,Basketball Court
2,Jordan Terminal 23,"145 W 32nd St (between 6th & 7th ave), New Yor...",40.749368,-73.990336,Basketball Court
3,Basketball Courts,"Canal St, New York, NY 10014",40.725597,-74.011405,Basketball Court
4,Basketball Gym At The Dylan,"5TH Ave (32nd St.), New York, NY 10118",40.746752,-73.985298,Basketball Court
5,Playground 9- Basketball Court/Sprinklers,"1ST Ave, New York, NY 10009",40.732120,-73.979747,Basketball Court
6,Robert Moses Basketball Courts,"42ND St, New York, NY 10017",40.748117,-73.968711,Basketball Court
7,UN Basketball Courts,"42nd Street and the FDR, New York, NY 10017",40.748117,-73.968711,Basketball Court
8,ABC Hoops NYC Midtown West Arena,"439 W 49th St, New York, NY 10019",40.763766,-73.990683,Basketball Court
9,BTSH Hockey League,"500 E 9th St (Ave. A & E 10th St.), New York, ...",40.724144,-73.981487,Basketball Court


In [35]:
def Partying_NY (NY_lat,NY_long,radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=13009,13007,13006,13015,13018&ll={NY_lat}%2C{NY_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    response = requests.get(url, headers=headers).json()

In [38]:
def everything_NYParty(one_element):
    address = one_element["location"]["formatted_address"]
    name = one_element["name"]
    category = one_element["categories"][0]["name"]
    latitude = one_element["geocodes"]["main"]["latitude"]
    longitude = one_element["geocodes"]["main"]["longitude"]
    dict5 = {"name": name, "address": address, "latitude": latitude, "longitude": longitude, "categories": category}
    return dict5

new_listNYParty = []
for i in response["results"]:
    new_listNYParty.append(everything_NYParty (i))

df_NY_Party = pd.DataFrame(new_listNYParty)
df_NY_Party
df_NY_Party.to_csv('./data/df_NY_party.csv')


In [39]:
df_NY_Party

,name,address,latitude,longitude,categories
0,Nubeluz,"25 W 28TH St (Broadway), New York, NY 10001",40.745630,-73.988879,Cocktail Bar
1,Patent Pending,"49 W 27th St (btwn Broadway & 6th Ave), New Yo...",40.745324,-73.990221,Cocktail Bar
2,The Flatiron Room,"37 W 26th St (btwn Broadway & 6th Ave), New Yo...",40.744565,-73.990366,Cocktail Bar
3,Oscar Wilde,"45 W 27th St, New York, NY 10001",40.745223,-73.989955,Cocktail Bar
4,The Smith,"1150 Broadway (at W 27th St), New York, NY 10001",40.744231,-73.988737,Cocktail Bar
5,Ms Kim’s,"6 W 28th St (btwn Broadway & 5th Ave), New Yor...",40.744732,-73.988188,Karaoke Bar
6,The Fleur Room,"105 W 28th St (6th Ave), New York, NY 10001",40.746491,-73.990965,Cocktail Bar
7,Cote,"16 W 22nd St (btw 5th & 6th Ave), New York, NY...",40.741195,-73.991281,Cocktail Bar
8,Thyme,"20 W 23rd St, New York, NY 10010",40.741755,-73.990517,Cocktail Bar
9,Il Fiorista Nomad,"17 W 26th St, New York, NY 10010",40.744210,-73.989522,Cocktail Bar


In [176]:
token1 = os.getenv("token1")

In [177]:
load_dotenv()

True

In [189]:
def vegan_NY (NY_lat, NY_long, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?categories=vegan&ll={NY_lat}%2C{NY_long}&radius={radius}"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    
    return vegan_places

In [190]:
vegan_places= vegan_NY (40.7444668, -73.9882868, 3500)
vegan_places

{'message': 'Invalid request token.'}

In [201]:
import folium
# Center de map to geolocation
mapa = folium.Map(location=[40.7451478, -73.9894417], zoom_start=12)

ruta_colegiosNY = './data/df_NY_SC.csv'
df_colegiosNY = pd.read_csv(ruta_colegiosNY)

pets_dataNY = './data/df_NY_Pet.csv'
df_petsNY = pd.read_csv(pets_dataNY)

starbucks_dataNY = './data/df_NY_ST.csv'
df_starbucksNY = pd.read_csv(starbucks_dataNY)

ruta_empresasNY = './data/df_NY.csv'
df_empresasNY = pd.read_csv(ruta_empresasNY)

partyNY = './data/df_NY_party.csv'
df_partyNY = pd.read_csv(partyNY)

airportsNY = './data/df_airports_ny.csv'
df_airportsNY = pd.read_csv(airportsNY)

basketNY='./data/df_NY_basket.csv'
df_basketNY=pd.read_csv(basketNY)

# Create markers for each category:

'''basket'''
for index, row in df_basketNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='orange', icon='hand-peace', prefix="fa"),
                  popup=popup_content).add_to(mapa)

'''schools'''
for index, row in df_colegiosNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='blue', icon='graduation-cap', prefix='fa'),
                  popup=popup_content).add_to(mapa)


'''restaurants'''
for index, row in df_restaurantesNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"

    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='green', icon='leaf'),
                  popup=popup_content).add_to(mapa)

'''companies > 1B, + design companies'''

for index, row in df_empresasNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='red', icon='building', prefix = "fa"),
                  popup=popup_content).add_to(mapa)

    
'''Pet Grooming service'''

for index, row in df_petsNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='pink', icon='paw', prefix = "fa"),
                  popup=popup_content).add_to(mapa)
    
'''Party'''
    
for index, row in df_partyNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Category: {row['categories']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='black', icon='beer', prefix = "fa"),
                  popup=popup_content).add_to(mapa)
    
    
'''Starbucks'''
for index, row in df_starbucksNY.iterrows():
    popup_content = f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}<br>Name: {row['name']}"
    folium.Marker([row['latitude'], row['longitude']],
                  icon=folium.Icon(color='purple', icon='coffee', prefix = "fa"),
                  popup=popup_content).add_to(mapa)

'''Airports'''
for index, row in df_airportsNY.iterrows():
    popup_content = f"Latitude: {row['latitude_deg']}<br>Longitude: {row['longitude_deg']}<br>Name: {row['name']}"
    folium.Marker([row['latitude_deg'], row['longitude_deg']],
                  icon=folium.Icon(color='cadetblue', icon='plane', prefix = "fa"),
                  popup=popup_content).add_to(mapa)   
'''vegan'''
vegan_restaurants = [
    {   "name": "Little Vet",
        "latitude": 40.742422,
        "longitude": -73.988154
    },
    {
        "name": "Coletta",
        "latitude": 40.740543,
        "longitude": -73.990823
    },
    {
        "name": "abcv",
        "latitude": 40.738373,
        "longitude": -73.990084
    },
    {
        "name": "Blossom",
        "latitude": 40.737486,
        "longitude": -73.988389
    },
    {
        "name": "Beyond Sushi - Union Square",
        "latitude": 40.734648,
        "longitude": -73.990401
    },
    {
        "name": "Candle 79",
        "latitude": 40.775811,
        "longitude": -73.958518
    },
    {
        "name": "The Butcher's Daughter - Nolita",
        "latitude": 40.722993,
        "longitude": -73.996122
    },
    {
        "name": "Divya's Kitchen",
        "latitude": 40.724202,
        "longitude": -73.987964
    },
    {
        "name": "Ladybird",
        "latitude": 40.732409,
        "longitude": -73.986839
    },
]

for restaurant in vegan_restaurants:
    popup_content = f"Name: {restaurant['name']}<br>Latitude: {restaurant['latitude']}<br>Longitude: {restaurant['longitude']}"
    folium.Marker([restaurant['latitude'], restaurant['longitude']],
                  icon=folium.Icon(color='green', icon='leaf'),
                  popup=popup_content).add_to(mapa)
mapa


Based on the initial criteria, and as no all requirements are present on the map, we dismiss New york